In [1]:
import os
os.environ['PYTHONHASHSEED'] = '1'
from numpy.random import seed
import random
random.seed(1)
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# TensorFlow wizardry
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.5

import sys
import math
import time
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from keras import backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

from cifar10_robust_models import Load_Madry_Model, cifar10_tf_robust_models

import numpy as np
import logging

class Autoencoder(object):
    def __init__(self,
                 dims,
                 activation='relu',
                 init='glorot_uniform',
                 verbose=1):
        '''
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
        The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        activation: activation, not applied to Input, last layer of the encoder, and Output layers
        '''
        self.dims = dims
        self.act = activation
        self.init = init
        self.verbose = verbose

    def build(self):
        """Fully connected auto-encoder model, symmetric.
        return:
            (ae_model, encoder_model), Model of autoencoder and model of encoder
        """
        input_img = Input(shape=(32, 32, 3))
        
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)
        self.encoded = encoded

        x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
        x = BatchNormalization()(x)
        x = UpSampling2D((2, 2))(x)

        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

        ae = Model(inputs=input_img, outputs=decoded, name='AE')
        encoder = Model(inputs=input_img, outputs=encoded, name='encoder')
        return ae, encoder

    def train_and_save(self, X,
                       weights_save_name,
                       lr=0.001,
                       batch_size=32,
                       epochs=250,
                       loss='mse'):
        if os.path.exists(weights_save_name):
            logging.info('weights file exists, no need to train pure AE')
        else:
            logging.debug(f'AE train_and_save lr: {lr}')
            logging.debug(f'AE train_and_save batch_size: {batch_size}')
            logging.debug(f'AE train_and_save epochs: {epochs}')

            verbose = self.verbose

            autoencoder, encoder = self.build()

            pretrain_optimizer = Adam(lr=lr)

            autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')

            utils.create_parent_folder(weights_save_name)

            mcp_save = ModelCheckpoint(weights_save_name,
                                    monitor='loss',
                                    save_best_only=True,
                                    save_weights_only=True,
                                    verbose=verbose,
                                    mode='min')

            hist = autoencoder.fit(X, X,
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=1,
                                callbacks=[mcp_save, logger.LoggingCallback(logging.debug)])

    def evaluate_quality(self, X_old, y_old, model_save_name):
        if not os.path.exists(model_save_name):
            self.train_and_save(X_old, model_save_name)

        K.clear_session()
        autoencoder, encoder = self.build()
        encoder.load_weights(model_save_name, by_name=True)
        logging.debug(f'Load weights from {model_save_name}')
        latent = encoder.predict(X_old)

        best_acc = 0
        best_n_init = 10
        num_classes = len(np.unique(y_old))
        logging.debug(f'KMeans k = {num_classes}')

        warnings.filterwarnings('ignore')

        for n_init in range(10, 110, 10):
            kmeans = KMeans(n_clusters=num_classes, n_init=n_init,
                            random_state=42, n_jobs=-1)
            y_pred = kmeans.fit_predict(latent)
            acc = utils.get_cluster_acc(y_old, y_pred)
            logging.debug(f'KMeans n_init: {n_init}, acc: {acc}')
            if acc > best_acc:
                best_n_init = best_n_init
                best_acc = acc
        logging.info(f'best accuracy of KMeans on latent data: {best_acc} with n_init {best_n_init}')
        return best_acc

Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [22]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train[0])

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
print(x_train.shape)
x_train = x_train[:30000]
y_train = y_train[:30000]
x_test = x_test[:30000]
y_test = y_test[:30000]


[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]
(50000, 32, 32, 3)


In [3]:
def batch_data(x_train, y_train,batch_size):
    batch_count = len(x_train)/batch_size
    shuffler = np.random.permutation(len(x_train))
    x_ret = x_train[shuffler]
    y_ret = y_train[shuffler]
    return batch_count, x_ret, y_ret

In [5]:
from keras.datasets import mnist
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
import keras
from cleverhans.utils_keras import KerasModelWrapper
import tensorflow as tf

from distutils.version import LooseVersion
if LooseVersion(keras.__version__) >= LooseVersion('2.0.0'):
	from keras.layers import Conv2D
else:
	from keras.layers import Convolution2D

# import argparse
import numpy as np
from sklearn.metrics import accuracy_score
import math
import os
from setup_cifar import CIFAR
# from tensorflow.python.platform import flags
# FLAGS = flags.FLAGS
IMAGE_ROWS = 32
IMAGE_COLS = 32
NUM_CHANNELS  = 3
NUM_CLASSES = 10

from keras.optimizers import SGD
from keras.utils import np_utils

def set_mnist_flags():
	# try:
	#     flags.DEFINE_integer('BATCH_SIZE', 64, 'Size of training batches')
	# except argparse.ArgumentError:
	#     pass
	flags.DEFINE_integer('BATCH_SIZE', 128, 'Size of training batches')
	flags.DEFINE_integer('NUM_CLASSES', 10, 'Number of classification classes')
	flags.DEFINE_integer('IMAGE_ROWS', 28, 'Input row dimension')
	flags.DEFINE_integer('IMAGE_COLS', 28, 'Input column dimension')
	flags.DEFINE_integer('NUM_CHANNELS', 1, 'Input depth dimension')


def data_mnist(one_hot=True):
	"""
	Preprocess MNIST dataset
	"""
	# the data, shuffled and split between train and test sets
	(X_train, y_train), (X_test, y_test) = mnist.load_data()

	y_train = y_train


	X_train = X_train.reshape(X_train.shape[0],
							  IMAGE_ROWS,
							  IMAGE_COLS,
							  NUM_CHANNELS)

	X_test = X_test.reshape(X_test.shape[0],
							IMAGE_ROWS,
							IMAGE_COLS,
							NUM_CHANNELS)

	X_train = X_train.astype('float32')
	X_test = X_test.astype('float32')
	X_train /= 255 - 0.5
	X_test /= 255 -0.5
	print('X_train shape:', X_train.shape)
	print(X_train.shape[0], 'train samples')
	print(X_test.shape[0], 'test samples')

	print("Loaded MNIST test data.")

	if one_hot:
		# convert class vectors to binary class matrices
		y_train = np_utils.to_categorical(y_train, NUM_CLASSES).astype(np.float32)
		y_test = np_utils.to_categorical(y_test, NUM_CLASSES).astype(np.float32)

	return X_train, y_train, X_test, y_test

# carlini model, use it as target model
def modelA(input_ph):
	model = Sequential()
	
	model.add(Conv2D(64, (3, 3),
							input_shape=(IMAGE_ROWS,
										 IMAGE_COLS,
										 NUM_CHANNELS)))
	model.add(Activation('relu'))
	model.add(Conv2D(64, (3, 3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Conv2D(128, (3, 3)))
	model.add(Activation('relu'))
	model.add(Conv2D(128, (3, 3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Flatten())
	model.add(Dense(256))
	model.add(Activation('relu'))
	model.add(Dropout(0.5))
	model.add(Dense(256))
	model.add(Activation('relu'))
	model.add(Dense(NUM_CLASSES))
	logits_tensor = model(input_ph)
	model.add(Activation('softmax'))

	return model, logits_tensor

def modelB(input_ph):
	model = Sequential()
	model.add(Dropout(0.2, input_shape=(IMAGE_ROWS,
										IMAGE_COLS,
										NUM_CHANNELS)))
	model.add(Convolution2D(64, (8, 8),
							subsample=(2, 2),
							border_mode='same'))
	model.add(Activation('relu'))

	model.add(Convolution2D(128, (6, 6),
							subsample=(2, 2),
							border_mode='valid'))
	model.add(Activation('relu'))

	model.add(Convolution2D(128, (5, 5),
							subsample=(1, 1)))
	model.add(Activation('relu'))

	model.add(Dropout(0.5))

	model.add(Flatten())
	model.add(Dense(NUM_CLASSES))
	logits_tensor = model(input_ph)
	model.add(Activation('softmax'))  #14

	return model, logits_tensor


def modelC(input_ph):
	model = Sequential()
	model.add(Convolution2D(128, (3, 3),
							border_mode='valid',
							input_shape=(IMAGE_ROWS,
										 IMAGE_COLS,
										 NUM_CHANNELS)))
	model.add(Activation('relu'))

	model.add(Convolution2D(64, (3, 3)))
	model.add(Activation('relu'))

	model.add(Dropout(0.25))

	model.add(Flatten())
	model.add(Dense(128))
	model.add(Activation('relu'))

	model.add(Dropout(0.5))
	model.add(Dense(NUM_CLASSES))
	logits_tensor = model(input_ph)
	model.add(Activation('softmax'))  #14

	return model, logits_tensor

def modelD(input_ph):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), padding='same',
					input_shape=(IMAGE_ROWS,
								 IMAGE_COLS,
								 NUM_CHANNELS)))
	model.add(Activation('relu'))
	model.add(Conv2D(32, (3, 3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))

	model.add(Conv2D(64, (3, 3), padding='same'))
	model.add(Activation('relu'))
	model.add(Conv2D(64, (3, 3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))

	model.add(Flatten())
	model.add(Dense(512))
	model.add(Activation('relu'))
	model.add(Dropout(0.5))
	model.add(Dense(NUM_CLASSES))
	logits_tensor = model(input_ph)
	model.add(Activation('softmax'))

	return model, logits_tensor

def modelE(input_ph):

	model = Sequential()
	model.add(Convolution2D(48, 3, 3, 
							border_mode='same', 
							input_shape=(IMAGE_ROWS,
										 IMAGE_COLS,
										 NUM_CHANNELS)))
	model.add(Activation('relu'))
	model.add(Convolution2D(48, 3, 3))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	model.add(Convolution2D(96, 3, 3, 
							border_mode='same'))
	model.add(Activation('relu'))
	model.add(Convolution2D(96, 3, 3))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	model.add(Convolution2D(192, 3, 3,
							border_mode='same'))
	model.add(Activation('relu'))
	model.add(Convolution2D(192, 3, 3))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	model.add(Flatten())
	model.add(Dense(512))
	model.add(Activation('relu'))
	model.add(Dropout(0.5))
	model.add(Dense(256))
	model.add(Activation('relu'))
	model.add(Dropout(0.5))
	model.add(Dense(NUM_CLASSES))
	logits_tensor = model(input_ph)
	model.add(Activation('softmax'))

	return model, logits_tensor


def model_cifar10(input_ph,type=1):
	"""
	Defines MNIST model using Keras sequential model
	"""

	models = [modelA, modelB, modelC, modelD, modelE]
	model = models[type]
	return model(input_ph)


def data_gen_mnist(X_train):
	datagen = ImageDataGenerator()

	datagen.fit(X_train)
	return datagen

# a unified framework for all mnist models


def main(path_dir,trainable,type,epochs):
	# load the data first
	x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3)) 
	data_augmentation = True
	data = CIFAR()
	X_train, Y_train, X_test, Y_test = data.train_data, data.train_labels, data.test_data, data.test_labels
	model_names = ['modelA','modelB','modelC','modelD','modelE']
	model, logits_tensor = model_cifar10(input_ph = x, type = type)
	path_dir = '' # put your file path for simple cifar10 models. THIS IS ONLY NEEDED WHEN TRAINING YOUR MODEL FROM SCRATCH
	# initiate RMSprop optimizer
	# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
	opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
	# Let's train the model using RMSprop
	model.compile(loss='categorical_crossentropy',
				optimizer=opt,
				metrics=['accuracy'])
	batch_size = 128
	if trainable:
		if not data_augmentation:
			print('Not using data augmentation.')
			model.fit(X_train, Y_train,
					batch_size=batch_size,
					epochs=epochs,
					validation_data=(X_test, Y_test),
					shuffle=True)
		else:
			print('Using real-time data augmentation.')
			# This will do preprocessing and realtime data augmentation:
			datagen = ImageDataGenerator(
				featurewise_center=False,  # set input mean to 0 over the dataset
				samplewise_center=False,  # set each sample mean to 0
				featurewise_std_normalization=False,  # divide inputs by std of the dataset
				samplewise_std_normalization=False,  # divide each input by its std
				zca_whitening=False,  # apply ZCA whitening
				zca_epsilon=1e-06,  # epsilon for ZCA whitening
				rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
				# randomly shift images horizontally (fraction of total width)
				width_shift_range=0.1,
				# randomly shift images vertically (fraction of total height)
				height_shift_range=0.1,
				shear_range=0.,  # set range for random shear
				zoom_range=0.,  # set range for random zoom
				channel_shift_range=0.,  # set range for random channel shifts
				# set mode for filling points outside the input boundaries
				fill_mode='nearest',
				cval=0.,  # value used for fill_mode = "constant"
				horizontal_flip=True,  # randomly flip images
				vertical_flip=False,  # randomly flip images
				# set rescaling factor (applied before any other transformation)
				rescale=None,
				# set function that will be applied on each input
				preprocessing_function=None,
				# image data format, either "channels_first" or "channels_last"
				data_format=None,
				# fraction of images reserved for validation (strictly between 0 and 1)
				validation_split=0.0)

			# Compute quantities required for feature-wise normalization
			# (std, mean, and principal components if ZCA whitening is applied).
			datagen.fit(X_train)

			# Fit the model on the batches generated by datagen.flow().
			model.fit_generator(datagen.flow(X_train, Y_train,
											batch_size=batch_size),
								epochs=epochs,
								validation_data=(X_test, Y_test),
								workers=4)
		# save model and weights
		model_path = path_dir + model_names[type]+'.h5'
		model.save(model_path)
		print('Saved trained model at %s ' % model_path)
	else:
		model = load_model(path_dir + model_names[type]+'.h5')

	temp = np.argmax(model.predict(X_test),axis=1)
	accuracy = accuracy_score(np.argmax(Y_test,axis=1), temp)
	print("oracle model accuracy:",accuracy)
	score = model.evaluate(X_test, Y_test, verbose=0)

	# Score trained model.
	scores = model.evaluate(X_test, Y_test, verbose=1)
	print('Test loss:', scores[0])
	print('Test accuracy:', scores[1])

if __name__ == '__main__':
	import argparse
	parser = argparse.ArgumentParser()
	parser.add_argument("--path_dir", type=str, default = "model/cifar10/",help="path to model")
	parser.add_argument("--type", type=int, help="model type", default=1)
	parser.add_argument("--epochs", type=int, default=50, help="number of epochs")
	parser.add_argument("--trainable", type=int, default = 1, help="decide if to train or directly load models")


	args = parser.parse_args()
	main(args.path_dir, args.trainable,args.type,args.epochs)

	

usage: ipykernel_launcher.py [-h] [--path_dir PATH_DIR] [--type TYPE]
                             [--epochs EPOCHS] [--trainable TRAINABLE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/chhabra4/.local/share/jupyter/runtime/kernel-f497e008-51de-4ee6-b7dc-9cca70a1e2f9.json


SystemExit: 2

/home/chhabra4/anaconda3/envs/attack/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
from keras.models import save_model
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

class cifar10_models_simple1(object):
	def __init__(self, sess, test_batch_size, type = 1,use_softmax = True, x = None, is_training=None,\
		 keep_prob=None,load_existing = False, model_name = 'modelA', loss = 'cw'):
		self.x = x
		self.sess = sess
		self.is_training = is_training
		self.keep_prob = keep_prob
		self.test_batch_size = test_batch_size
		self.model_name = model_name
		self.old_vars = set(tf.global_variables())
		if load_existing:
			save_dir = 'CIFAR10_models/Normal_simple_models' # TODO: put your own ROOT directory of simple cifar10 models
			filepath = os.path.join(save_dir, model_name+'.h5')
			filepath = 'CIFAR10_models/Normal_simple_models/simple2/saved'
			with tf.variable_scope('cifar10_model_simple', reuse=tf.AUTO_REUSE):
				model = load_model(filepath)
				#model._make_predict_function()
				self.model = model
				model = KerasModelWrapper(model)
				self.predictions = model.get_logits(self.x)
		else:
			with tf.variable_scope('cifar10_model_simple', reuse=tf.AUTO_REUSE):
				model, preds = model_cifar10(input_ph = x, type = type)
				self.model = model
				self.predictions = preds
		self.new_vars = set(tf.global_variables())
		self.var_list = self.new_vars - self.old_vars
		self.probs = tf.nn.softmax(logits = self.predictions)
		self.eval_preds = tf.argmax(self.predictions, 1)
		self.y_target = tf.placeholder(tf.int64, shape=None) # tensor.shape (?,)
		self.eval_percent_adv = tf.equal(self.eval_preds, self.y_target) # one-to-one comparison


	def load_weights(self):
		#print(mapping2)
		with tf.variable_scope('cifar10_model_simple', reuse=tf.AUTO_REUSE):
			save_dir = 'CIFAR10_models/Normal_simple_models' # TODO: put your own ROOT directory of simple cifar10 models
			filepath = 'CIFAR10_models/Normal_simple_models/simple2/saved'
			self.model.load_weights(filepath)
			#model._make_predict_function()
			#self.model = model
			#model = KerasModelWrapper(model)
			#self.predictions = model.get_logits(self.x)
			#self.probs = tf.nn.softmax(logits = self.predictions)
			#output = tf.identity(self.probs)
			#return output
            

	def save_everything(self):
		h = self.model.save('CIFAR10_models/Normal_simple_models/simple2/saved')
		print(h)

	def calcu_acc(self,data,lab):
		batch_size = self.test_batch_size
		#numpy operation to get prediction value
		corr_preds = 0
		num_batches = int(math.ceil(len(data) / batch_size))
		for ibatch in range(num_batches):
			bstart = ibatch * batch_size
			bend = min(bstart + batch_size, len(data))
			data_batch = data[bstart:bend,:]
			lab_batch = lab[bstart:bend]
			# acc calculation
			preds = self.sess.run(self.predictions,feed_dict = {self.x:data_batch})
			corr_preds += np.sum(np.argmax(lab_batch,axis = 1) == np.argmax(preds,axis = 1))
		return corr_preds/len(data)

	def predict_prob(self,data):
		batch_size = self.test_batch_size
		probs = []
		num_batches = int(math.ceil(len(data) / batch_size))
		for ibatch in range(num_batches):
			bstart = ibatch * batch_size
			bend = min(bstart + batch_size, len(data))
			data_batch = data[bstart:bend,:]
			# acc calculation
			prob = self.sess.run(self.probs,feed_dict = {self.x:data_batch})
			probs.extend(prob)
		return np.array(probs) 
	def pred_class(self,data):
		preds = self.predict_prob(data)
		labels = np.argmax(preds,axis = 1)
		return labels

	def eval_adv(self, adv, target_class):
		feed_dict = {self.x: adv, self.y_target: target_class}
		padv = self.sess.run(self.eval_percent_adv, feed_dict=feed_dict)
		return padv

	def get_loss(self, data, labels,class_num = 10):
		if len(labels.shape) == 1:
			labels = np_utils.to_categorical(labels, class_num)
		feed_dict = {self.x: data, self.y: labels}
		loss_val = self.sess.run(self.loss, feed_dict = feed_dict)
		return loss_val 

In [6]:
import importlib

tf.reset_default_graph()

sess = tf.Session()
K.set_session(sess)

class_num = 10
image_size = 32
num_channels = 3
test_batch_size = 10
x = tf.placeholder(tf.float32, shape=(None, image_size, image_size, num_channels))
y = tf.placeholder(tf.float32, shape=(None, class_num))
input_ = x

target_model2 = cifar10_models_simple1(sess,10, 0, use_softmax=True,x = input_, 
                                                              load_existing=False,model_name='modelA')

output = tf.identity(target_model2.probs)
# for i in tf.trainable_variables():
#     mapping2[i.name[:-2]] = i
#     #print(i.name)

# print(mapping2)
with sess as sess:
    sess.run(tf.global_variables_initializer())
    target_model2.load_weights()
    #yb = sess.run([target_model2.predictions],feed_dict = {input_ : x_train[:10]})
    #print(yb)
    lb = sess.run([output],feed_dict = {input_ : x_train[:10]})
    print(lb)
    print(np.argmax(lb, axis=2))
    yb,tb = sess.run([target_model2.probs,output],feed_dict = {input_ : x_train[:10]})
    print(yb)
    print(tb)
    pro = target_model2.pred_class(x_train[:20])
    print(pro)
    print(y_train[:20])
#     target_model3.load_weights()
#     pro = target_model3.pred_class(x_train[20:30])
#     print(pro)
#     print(y_train[20:30])
#     target_model2.save_everything()

#tf.train.Saver(var_list=mapping).save(sess, 'CIFAR10_models/Normal_simple_models/simple/cp1')
    #target_model2.save_everything()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[array([[1.01481592e-02, 9.02265951e-04, 5.56485504e-02, 9.61109921e-02,
        2.54448038e-02, 1.95823777e-02, 7.71196842e-01, 3.82890739e-03,
        1.62217598e-02, 9.15372162e-04],
       [4.37106501e-04, 2.61574751e-03, 3.96338919e-06, 2.32017446e-05,
        5.25345740e-08, 2.51373467e-06, 6.52457555e-09, 8.40115172e-06,
        4.35338030e-03, 9.92555559e-01],
       [7.46670961e-02, 6.41951337e-04, 5.83638554e-04, 7.27261358e-05,
        1.77397320e-04, 2.29943780e-05, 3.02832450e-05, 5.57952677e-04,
        8.39151721e-03, 9.14854348e-01],
       [1.12772236e-08, 4.33854552e-09, 9.31090108e-06, 4.59966623e-06,
        9.99934316e-01, 1.99548831e-05, 5.64594075e-06, 2.62103204e-05,
        1.93373006e-09, 2.61356501e-08],
       [5.88382045e-13, 9.99999881e-01, 6.88999316e-20, 1.76768447e-17,


In [38]:
from cifar10_robust_models import Load_Madry_Model, cifar10_tf_robust_models
from cifar10_simple_models import cifar10_models_simple

tvars_vals = []

class ContrastiveAE(object):
    def __init__(self, optimizer, lr, dims = 1, verbose=1):
        self.dims = dims
        self.optimizer = optimizer(lr)
        self.verbose = verbose
        print("init")
    
    def visualize(self, x_train, y_train):
        tf.reset_default_graph()
        
        ae = Autoencoder(self.dims)
        ae_model, encoder_model = ae.build()
        input_ = ae_model.get_input_at(0)
        noise = tf.random_normal(shape=tf.shape(input_), mean=0.0, stddev=0.02, dtype=tf.float32)
        noise_abs =tf.math.abs(noise)
        noise_img = tf.add(input_, noise)
        noise_img_clipped = tf.clip_by_value(noise_img, clip_value_min=0, clip_value_max=1)
        aeoutput = ae_model(noise_img_clipped)
        
        with tf.Session(config=config) as sess:
            ae_model.load_weights('./query-blinding-6-just-weights')
            
            feed_dict = {
                input_: x_train
            }
            output, input_with_noise, noise_vis = sess.run([aeoutput, noise_img_clipped, noise_abs], feed_dict=feed_dict)
            #output = np.array(output)
#             print(output[0].shape)
#             print(output[0][0].shape)
            #print(output)
            img = Image.fromarray((output[0]*255).astype('uint8'), 'RGB')
            img.save('aeoutput-noise.png')
            img.show()
            img = Image.fromarray((input_with_noise[0]*255).astype('uint8'), 'RGB')
            img.save('aeinput-noise.png')
            img.show()
            img = Image.fromarray((noise_vis[0]*255).astype('uint8'), 'RGB')
            img.save('noise.png')
            img.show()
            #ae_model.save("saved_autoencoder_qb")
        
    
    def train(self, x_train, y_train,
              lambda_1, batch_size, epochs, similar_ratio, margin,
              weights_save_name, display_interval):
        """Train an autoencoder with standard mse loss + contrastive loss.
        Arguments:
            X_train {numpy.ndarray} -- feature vectors of the training data
            y_train {numpy.ndarray} -- ground-truth labels of the training data
            lambda_1 {float} -- balance factor for the autoencoder reconstruction loss and contrastive loss
            batch_size {int} -- number of samples in each batch (note we only use **half of batch_size**
                                from the training data).
            epochs {int} -- No. of maximum epochs.
            similar_ratio {float} -- ratio of similar samples, use 0.25 for now.
            margin {float} -- the hyper-parameter m.
            weights_save_name {str} -- file path to save the best weights files.
            display_interval {int} -- print traning logs per {display_interval} epoches
        """
        print("hello")
        if False:
            logging.info('weights file exists, no need to train contrastive AE')
        else:
            tf.reset_default_graph()
            
            sess = tf.Session(config=config)
            K.set_session(sess)
           
            lambda_1_tensor = tf.placeholder(tf.float32)
            ae = Autoencoder(self.dims)
            ae_model, encoder_model = ae.build()

            input_ = ae_model.get_input_at(0)
            aclabels = tf.placeholder(tf.int32, [None, 1])
            noise = tf.random_normal(shape=tf.shape(input_), mean=0.0, stddev=0.02, dtype=tf.float32)
            noise_img = tf.add(input_, noise)
            noise_img = tf.clip_by_value(noise_img, clip_value_min=0, clip_value_max=1)
            print(noise_img.shape)
            noise_second = tf.random_normal(shape=tf.shape(input_), mean=0.0, stddev=0.02, dtype=tf.float32)
            noise_img_second = tf.add(input_, noise_second)
            noise_img_second = tf.clip_by_value(noise_img_second, clip_value_min=0, clip_value_max=1)


            aeoutput = ae_model(noise_img)
            aeoutput_second = ae_model(noise_img_second)
            ae_loss = tf.keras.losses.MSE(noise_img, ae_model(noise_img)) # ae.out equals ae_model(input_)
            ae_loss = tf.reduce_mean(ae_loss)
            
            model_dir = "./"
#             target_model2 = Load_Madry_Model(tf.Session(config=config), model_dir, noise_img, bias = 0.5, scale = 255)
#             target_model3 = Load_Madry_Model(tf.Session(config=config), model_dir, aeoutput, bias = 0.5, scale = 255)
            t_m_o = tf.placeholder(tf.float32, shape=(None, 10)) 
            target_model2 = cifar10_models_simple1(sess,10, 0, use_softmax=True,x = noise_img, 
                                                              load_existing=False,model_name='modelA')
            target_model3 = cifar10_models_simple1(sess,10, 0, use_softmax=True,x = aeoutput, 
                                                              load_existing=False,model_name='modelA')            
            #print(target_model2.var_list)
            output = tf.identity(target_model2.probs, name='my_output')
            output2 = tf.identity(target_model3.probs, name='my_output2')
            
            aeoutput_mul = aeoutput*255
            aeoutput_second_mul = aeoutput_second*255
            l2_norm = tf.norm(aeoutput_second_mul - aeoutput_mul, ord='euclidean', axis=[-2,-1])
            l2_norm_test = l2_norm
            l2_norm = tf.pow(l2_norm, 2)
            l2_norm = tf.math.minimum(l2_norm, 20)
            
            
            loss_crossentropy = tf.keras.losses.sparse_categorical_crossentropy(aclabels, output2)
            loss_crossentropy = tf.reduce_mean(loss_crossentropy)
            loss = loss_crossentropy - l2_norm
            
            
            train_op = self.optimizer.minimize(loss, var_list=list(set(tf.trainable_variables()) - target_model2.var_list - target_model3.var_list))
            print(list(set(tf.trainable_variables()) - target_model2.var_list - target_model3.var_list))
            # Start training
            with sess as sess:
                loss_batch, aux_batch, ce_batch = [], [], []
                contrastive_loss_batch, ae_loss_batch = [], []
                
                sess.run(tf.global_variables_initializer())
                target_model2.load_weights()
                target_model3.load_weights()
                min_loss = np.inf 
                
               
                # epoch training loop
                for epoch in range(250):
                    epoch_time = time.time()
                    # split data into batches
                    batch_size = 100
                    batch_count, batch_x, batch_y = batch_data(x_train, y_train, batch_size)
                    batch_count = int(batch_count)
                    print("BC: %d" % batch_count)
                    # batch training loop
                    
                    for b in range(batch_count):
                        logging.debug(f'b: {b}')
                        #add random noise to x_train (start with 0.01 first)
                        feed_dict = {
                            input_: batch_x[b*batch_size:b*batch_size + batch_size],
                            lambda_1_tensor: lambda_1,
                            aclabels: batch_y[b*batch_size:b*batch_size + batch_size],
                        }
#                         loss1, _, ae_loss1, \
#                             encoded1, labelled = sess.run([loss, train_op, ae_loss, ae.encoded, target_model2.model.softmax_pred], feed_dict=feed_dict)
                        labelled, labelled2,loss1,ae_loss1,loss_ce,_ = sess.run([output,output2,loss,l2_norm,loss_crossentropy,train_op], feed_dict=feed_dict)
                        #print(np.argmax(labelled, axis=1))
                        #print(np.argmax(labelled2, axis=1))
                        #print(labelled)
                        #print(*y_train)
                        
                        loss_batch.append(loss1)
                        ae_loss_batch.append(ae_loss1)
                        ce_batch.append(loss_ce)

                    if math.isnan(np.mean(loss_batch)):
                        logging.error('NaN value in loss')
                    
                    current_loss = np.mean(loss_batch)
                    print(f'Epoch {epoch}: loss {current_loss} -- ' + \
                                    f'ae {np.mean(ae_loss_batch)} -- ' + \
                                    f'ae {np.mean(ce_batch)} -- ' + \
                                    f'time {time.time() - epoch_time}')
                print("saving weights")
                ae_model.save('./query-blinding-6-whole-model')
                ae_model.save_weights('./query-blinding-6-just-weights')
                

In [37]:
OPTIMIZER = tf.train.AdamOptimizer
cae = ContrastiveAE(OPTIMIZER, 0.01)
print(len(x_train))
x_train_t = x_train[:8000]
y_train_t = y_train[:8000]
cae.train(x_train_t, y_train_t,
                1e-1, 1, 1, 1, 1,
                1, 20)

init
30000
hello
(?, 32, 32, 3)
[<tf.Variable 'batch_normalization_4/moving_variance:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization_1/gamma:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 64, 16) dtype=float32_ref>, <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>, <tf.Variable 'batch_normalization_2/beta:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'conv2d_5/bias:0' shape=(3,) dtype=float32_ref>, <tf.Variable 'batch_normalization_3/gamma:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'conv2d_5/kernel:0' shape=(3, 3, 32, 3) dtype=float32_ref>, <tf.Variable 'batch_normalization_3/beta:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'batch_normalization_1/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv2d_3/bias:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'batch_normalization_3/moving_mean:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'batch_normalization_2/gamma:0' shape=(16,) dtype=float32

In [88]:
from cifar10_robust_models import Load_Madry_Model, cifar10_tf_robust_models

tvars_vals = []

class ContrastiveAE(object):
    def __init__(self, optimizer, lr, dims = 1, verbose=1):
        self.dims = dims
        self.optimizer = optimizer(lr)
        self.verbose = verbose
        print("init")
    
    def train(self, X_train, y_train, actual_labels,
              lambda_1, batch_size, epochs, similar_ratio, margin,
              weights_save_name, display_interval):
        """Train an autoencoder with standard mse loss + contrastive loss.
        Arguments:
            X_train {numpy.ndarray} -- feature vectors of the training data
            y_train {numpy.ndarray} -- ground-truth labels of the training data
            lambda_1 {float} -- balance factor for the autoencoder reconstruction loss and contrastive loss
            batch_size {int} -- number of samples in each batch (note we only use **half of batch_size**
                                from the training data).
            epochs {int} -- No. of maximum epochs.
            similar_ratio {float} -- ratio of similar samples, use 0.25 for now.
            margin {float} -- the hyper-parameter m.
            weights_save_name {str} -- file path to save the best weights files.
            display_interval {int} -- print traning logs per {display_interval} epoches
        """
        print("hello")
        if False:
            logging.info('weights file exists, no need to train contrastive AE')
        else:
            tf.reset_default_graph()

           
            lambda_1_tensor = tf.placeholder(tf.float32)
            ae = Autoencoder(self.dims)
            ae_model, encoder_model = ae.build()

            input_ = ae_model.get_input_at(0)
            labels = ae_model.get_input_at(0)
            aclabels = tf.placeholder(tf.float32, [None, 1])
            # add loss function -- for efficiency and not doubling the network's weights, we pass a batch of samples and
            # make the pairs from it at the loss level.
#             left_p = tf.convert_to_tensor(list(range(0, int(batch_size / 2))), np.int32)
#             right_p = tf.convert_to_tensor(list(range(int(batch_size / 2), batch_size)), np.int32)

#             # left_p: indices with all the data in this batch, right_p: half with similar data compared to left_p, half with dissimilar data compared to left_p
#             # if batch_size = 16 (but only using 8 samples in this batch):
#             # e.g., left_p labels: 1, 2, 4, 8 | 2, 3, 5, 6
#             #      right_p labels: 1, 2, 4, 8 | 3, 4, 1, 7
#             # check whether labels[left_p] == labels[right_p] for each element
#             is_same = tf.cast(tf.equal(tf.gather(labels, left_p), tf.gather(labels, right_p)), tf.float32)
#             # NOTE: add a small number like 1e-10 would prevent tf.sqrt() to have 0 values, further leading gradients and loss all NaN.
#             # check: https://stackoverflow.com/questions/33712178/tensorflow-nan-bug
#             dist = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(tf.gather(ae.encoded, left_p), tf.gather(ae.encoded, right_p))), 1) + 1e-10) # ||zi - zj||_2
#             contrastive_loss = tf.multiply(is_same, dist) # y_ij = 1 means the same class.
#             contrastive_loss = contrastive_loss + tf.multiply((tf.constant(1.0) - is_same), tf.nn.relu(margin - dist))  # as relu(z) = max(0, z)
#             contrastive_loss = tf.reduce_mean(contrastive_loss)

            aeoutput = ae_model(input_)
            ae_loss = tf.keras.losses.MSE(input_, ae_model(input_)) # ae.out equals ae_model(input_)
            ae_loss = tf.reduce_mean(ae_loss)
            
            model_dir = "./"
#             target_model2 = Load_Madry_Model(tf.Session(config=config), model_dir, input_, bias = 0.5, scale = 255)
#             target_model3 = Load_Madry_Model(tf.Session(config=config), model_dir, aeoutput, bias = 0.5, scale = 255)
            
#             output = tf.identity(target_model2.model.softmax_pred, name='my_output')
#             output2 = tf.identity(target_model3.model.softmax_pred, name='my_output2')
            
#             cce = tf.keras.losses.CategoricalCrossentropy()
#             loss = cce(aclabels, output2) + ae_loss

            train_op = self.optimizer.minimize(ae_loss, var_list=tf.trainable_variables())

            # Start training
            with tf.Session(config=config) as sess:
                loss_batch, aux_batch = [], []
                contrastive_loss_batch, ae_loss_batch = [], []
                
                sess.run(tf.global_variables_initializer())
                #target_model2.load_weights()
                min_loss = np.inf
                
                #print(list(set(tf.trainable_variables()) - target_model2.var_list - target_model3.var_list))    
                
                # epoch training loop
                for epoch in range(100):
                    epoch_time = time.time()
                    # split data into batches
#                     batch_count, batch_x, batch_y = data.epoch_batches(X_train, y_train,
#                                         %%!                            batch_size,
#                                                                     similar_ratio)
#                     # batch training loop
                    
                    for b in range(1):
                        logging.debug(f'b: {b}')
                    
                        feed_dict = {
                            input_: x_train,
                            labels: y_train,
                            lambda_1_tensor: lambda_1,
                            aclabels: actual_labels,
                        }
#                         loss1, _, ae_loss1, \
#                             encoded1, labelled = sess.run([loss, train_op, ae_loss, ae.encoded, target_model2.model.softmax_pred], feed_dict=feed_dict)
                        ae_loss1, _ = sess.run([ae_loss, train_op], feed_dict=feed_dict)
                        loss1 = 0
                        #ae_loss1 = 0
                        
                        #print(np.argmax(labelled, axis=1))
                        #print(np.argmax(labelled2, axis=1))
                        #print(labelled)
                        print(f'loss1: {ae_loss1},  aux1: -')
#                         logging.debug(f'contrastive: {contrastive_loss1}, ae: {ae_loss1}')
#                         logging.debug(f'epoch-{epoch} dist1[left]: {dist1[0:batch_size // 4]}')
#                         logging.debug(f'epoch-{epoch} dist1[right]: {dist1[batch_size // 4:]}')
        
                        loss_batch.append(loss1)
#                         aux_batch.append(aux1)
#                         contrastive_loss_batch.append(contrastive_loss1)
                        ae_loss_batch.append(ae_loss1)

                    if math.isnan(np.mean(loss_batch)):
                        logging.error('NaN value in loss')
                    
                    current_loss = np.mean(loss_batch)
#                     #print(f'Epoch {epoch}: loss {current_loss} -- ' + \
#                                     f'ae {np.mean(ae_loss_batch)} -- ' + \
#                                     f'time {time.time() - epoch_time}')
                    # print logs each xxx epoch
#                     if epoch % display_interval == 0:
#                         current_loss = np.mean(loss_batch)
#                         logging.info(f'Epoch {epoch}: loss {current_loss} -- ' + \
#                                     f'contrastive {np.mean(contrastive_loss_batch)} -- ' + \
#                                     f'ae {np.mean(ae_loss_batch)} -- ' + \
#                                     f'pairs {np.mean(np.sum(np.mean(aux_batch)))} : ' + \
#                                     f'{np.mean(np.sum(1-np.mean(aux_batch)))} -- ' + \
#                                     f'time {time.time() - epoch_time}')
#                         loss_batch, aux_batch = [], []
#                         contrastive_loss_batch, ae_loss_batch = [], []

#                         # save best weights
#                         if current_loss < min_loss:
#                             logging.info(f'updating best loss from {min_loss} to {current_loss}')
#                             min_loss = current_loss
#                             ae_model.save_weights(weights_save_name)

In [40]:
from PIL import Image

OPTIMIZER = tf.train.AdamOptimizer
cae = ContrastiveAE(OPTIMIZER, 0.01)
x_train_vis = x_train[2:3]
cae.visualize(x_train_vis, y_train)

init
